In [2]:
import import_ipynb
import os

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import date

import scipy.stats as stats
from scipy.stats import chi2_contingency
import seaborn as sns

In [3]:
import import_data

importing Jupyter notebook from import_data.ipynb
Connected to snowflake. Now read in query = open('/Folder Directory/sql_query_of_data.sql','r').read()


### Script Contents

#### 1. Set Input Variables
Designate input variables for now, optimizely_dashboard_results table needed to automate
#### 2. Import Data
This imports data from multiple optimizely and datawarehouse tables. Takes awhile to run.
#### 3. Create summary table by experiment + variation 
The "summary" output table will contain total guid counts, conversion counts, and conversion rate for both optimizely ("opt_") and our datawarehouse ("dw_").
#### 4. Fishers Test
Outputs the contingency table (variant x conversion), odds ratio, and p-value.

### 1. Set Input Variables

In [ ]:
# Input variables
experiment_id = '20053754433'
platform = 'both' # web, native, or both
conv_event= 'decorated_invoice_submitted'

like_experiment_filter = 'AND (EXPERIMENT_ID LIKE \'%'+experiment_id+'%\')'
equals_experiment_filter = 'AND (EXPERIMENT_ID = \''+experiment_id+'\')'

### 2. Import data

#### Import DASHBOARD_RESULTS

In [5]:
# Working with Manasi to get this table up to date

# query =\
# """
# SELECT 		EXPERIMENT_ID AS "experiment_id"
# 		,EXPERIMENT_NAME AS "experiment_name"
# 		,VARIATION_ID AS "variation_id"
# 		,VARIATION_NAME AS "variation_name"
# 		,MAX(IS_BASELINE_VARIATION) AS "is_baseline_variation"
# --			,MIN(START_TIME_UTC),MAX(END_TIME_UTC),COUNT(DISTINCT VARIATION_ID),MAX(VARIATION_ID),MIN(VARIATION_ID)
# FROM 		PUBLIC.OPTIMIZELY_DASHBOARD_RESULTS 
# WHERE 		EXPERIMENT_NAME LIKE 'growth-%'
# GROUP BY 	1,2,3,4
# HAVING 		DATEDIFF(DD,MAX(END_TIME_UTC),CURRENT_DATE) BETWEEN 0 AND 1
# ORDER BY 	MAX(END_TIME_UTC) DESC, EXPERIMENT_NAME DESC
# """
# db_results = import_data.query_to_df(query)
# db_results

#### Import IMPR_EVENTS

In [6]:
# May not be necessary to pull this in. 1:1 with ENR_DECI table

#### Import ENR_DECI

In [117]:
# Test enr_deci query for now
query =\
"""
SELECT	 VISITOR_ID AS "visitor_id"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,ENVIRONMENT AS "environment"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_DECI 
WHERE ENVIRONMENT='production'
"""\
+equals_experiment_filter
# WHERE CREATED_AT >= """+'\''+dt_from.strftime("%Y-%m-%d")+'\''

enr_deci = import_data.query_to_df(query)
enr_deci.head()


SELECT	 VISITOR_ID AS "visitor_id"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,ENVIRONMENT AS "environment"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_DECI 
WHERE ENVIRONMENT='production'
AND (EXPERIMENT_ID = '20053754433')


,visitor_id,experiment_id,variation_id,guid,environment,timestamp
0,94c03785-dedf-499c-b1a5-97b3067d12dd,20053754433,20057713467,9bd65798-0066-4bcf-afeb-65a024038cd8,production,2021-03-28 17:10:39.647
1,97467f19-20df-9c20-a6c2-c32a4c48ace1,20053754433,20041984016,6c1e6dae-16e8-47aa-d2be-bd44606cedc3,production,2021-03-27 09:33:20.086
2,1a499ecd-7998-4559-a526-d5708925470d,20053754433,20041984016,1a499ecd-7998-4559-a526-d5708925470d,production,2021-03-27 01:43:00.835
3,47f60dfb-2730-4e95-b1c3-1c344fe4a9a3,20053754433,20057713467,47f60dfb-2730-4e95-b1c3-1c344fe4a9a3,production,2021-03-27 00:14:40.617
4,564976a4-100b-448e-aa30-36d4d68bc42b,20053754433,20041984016,33fd7d3f-abc8-e896-4137-0a1b12f3a159,production,2021-03-28 21:36:08.649


#### Import ENR_CONV

In [9]:
# Test enr_conv query for now
query =\
"""
SELECT	 VISITOR_ID AS "visitor_id"
        ,EVENT_NAME AS "event_name"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_CONV
WHERE ENVIRONMENT='production'
"""\
+like_experiment_filter

enr_conv=import_data.query_to_df(query)
enr_conv.head()


SELECT	 VISITOR_ID AS "visitor_id"
        ,EVENT_NAME AS "event_name"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_CONV
WHERE ENVIRONMENT='production'
AND (EXPERIMENT_ID LIKE '%20053754433%')


#### Import DW_EVENTS

In [135]:
# Get the start date of the experiment to load into query
first_timestamp = enr_deci.timestamp.min().strftime('%Y-%m-%d')
last_timestamp = enr_deci.timestamp.max().strftime('%Y-%m-%d')

web_query =\
"""
SELECT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS
"""+'WHERE "created_at" BETWEEN \''+first_timestamp+'\' AND \''+last_timestamp+'\''\

native_query =\
"""
SELECT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'native' AS "platform"
FROM PUBLIC.REACT_NATIVE_EVENTS
"""+'WHERE "created_at" BETWEEN \''+first_timestamp+'\' AND \''+last_timestamp+'\''\

invoices_query =\
"""
SELECT	posters.GUID AS "guid",
		'decorated_invoice_submitted' AS "event",
    	job_invoices.SUBMITTED_STATE_AT AS "submitted_state_date",
    	client_applications.APPLICATION AS "application"
FROM	JOBS AS jobs
		INNER JOIN JOB_INVOICES AS job_invoices ON jobs.ID = job_invoices.JOB_ID
		LEFT JOIN USERS AS posters ON jobs.POSTER_ID = posters.ID
		LEFT JOIN CLIENT_APPLICATIONS AS client_applications ON jobs.CREATED_CLIENT_APPLICATION_ID = client_applications.ID
WHERE 	(NOT (jobs.poster_rabbit_relationship_id::boolean) OR (jobs.poster_rabbit_relationship_id::boolean) IS NULL) 
		AND (jobs.V2_ID IS NULL)
		AND ((NOT (jobs.suppress_level >= 1000 ) OR (jobs.suppress_level >= 1000 ) IS NULL) AND (( jobs.FIXUP_KEY  IS NULL))) 
		AND ((NOT (posters.real_user_id is not null ) OR (posters.real_user_id is not null ) IS NULL) AND (NOT (posters.admin::boolean ) OR (posters.admin::boolean ) IS NULL) AND ((NOT (posters.internal::boolean ) OR (posters.internal::boolean ) IS NULL) AND (NOT (posters.suppress_level >= 1000 ) OR (posters.suppress_level >= 1000 ) IS NULL)))
"""+'AND "submitted_state_date" BETWEEN \''+first_timestamp+'\' AND \''+last_timestamp+'\''\

if platform == 'web':
    dw_query = web_query+'\nUNION'+invoices_query
elif platform == 'native':
    dw_query = native_query+'\nUNION'+invoices_query
elif platform == 'both':
    dw_query = web_query+'UNION'+native_query+'UNION'+invoices_query


SELECT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS
WHERE "created_at" BETWEEN '2021-03-16' AND '2021-04-21'UNION
SELECT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'native' AS "platform"
FROM PUBLIC.REACT_NATIVE_EVENTS
WHERE "created_at" BETWEEN '2021-03-16' AND '2021-04-21'UNION
SELECT	posters.GUID AS "guid",
		'decorated_invoice_submitted' AS "event",
    	job_invoices.SUBMITTED_STATE_AT AS "submitted_state_date",
    	client_applications.APPLICATION AS "application"
FROM	JOBS AS jobs
		INNER JOIN JOB_INVOICES AS job_invoices ON jobs.ID = job_invoices.JOB_ID
		LEFT JOIN USERS AS posters ON jobs.POSTER_ID = posters.ID
		LEFT JOIN CLIENT_APPLICATIONS AS client_applications ON jobs.CREATED_CLIENT_APPLICATION_ID = client_applications.ID
WHERE 	(NOT (jobs.poster_rabbit_relationship_id::boolean) OR (jobs.poster_rabbit_relationship_id::boolean) IS NULL) 
		A

In [ ]:
print(platform)
dw_events=import_data.query_to_df(dw_query)
dw_events.head()


SELECT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS
WHERE "created_at" BETWEEN '2021-03-16' AND '2021-04-21'UNION
SELECT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'native' AS "platform"
FROM PUBLIC.REACT_NATIVE_EVENTS
WHERE "created_at" BETWEEN '2021-03-16' AND '2021-04-21'UNION
SELECT	posters.GUID AS "guid",
		'decorated_invoice_submitted' AS "event",
    	job_invoices.SUBMITTED_STATE_AT AS "submitted_state_date",
    	client_applications.APPLICATION AS "application"
FROM	JOBS AS jobs
		INNER JOIN JOB_INVOICES AS job_invoices ON jobs.ID = job_invoices.JOB_ID
		LEFT JOIN USERS AS posters ON jobs.POSTER_ID = posters.ID
		LEFT JOIN CLIENT_APPLICATIONS AS client_applications ON jobs.CREATED_CLIENT_APPLICATION_ID = client_applications.ID
WHERE 	(NOT (jobs.poster_rabbit_relationship_id::boolean) OR (jobs.poster_rabbit_relationship_id::boolean) IS NULL) 
		A

### 3. Create summary table by experiment + variation

In [53]:
def count_conversions(exp_id,var_id,evt_name):
    conv_in_exp_var = enr_conv[\
                           (enr_conv['experiment_id'].str.find(exp_id)!=-1)\
                           & (enr_conv['variation_id'].str.find(var_id)!=-1)\
                            & (enr_conv['event_name'] == evt_name)\
                          ]
    conv_by_event = conv_in_exp_var.groupby('event_name').agg({'guid':'nunique'}).reset_index()
    return(conv_by_event.guid)

In [73]:
def get_dw_info(exp_id,var_id,evt_name): # Pass in 3 vars from summary table
    guids = enr_deci[(enr_deci['experiment_id'].str.find(exp_id)!=-1) & (enr_deci['variation_id'].str.find(var_id)!=-1)]
    guids = guids.guid.unique() # num guids in the experiment+variation
    
    dw = dw_events[(dw_events.created_at>=first_timestamp) & (dw_events.created_at<=last_timestamp)\
                   & (dw_events.guid.isin(guids))] # dw events from guids in the experiment+variation
    
    guid_ct = dw.guid.nunique()
    conv_ct = dw[dw.client_publish_type==evt_name]
    conv_ct = conv_ct.guid.nunique()
    first_date = dw.created_at.min()
    last_date = dw.created_at.max()
    
    print('get_dw_info_complete')
    
    return guid_ct,conv_ct,first_date,last_date # Return event info from datawarehouse

In [101]:
opt_summary = enr_deci.groupby(['experiment_id','variation_id']).agg({'timestamp':['min','max'],'guid':'nunique'}).reset_index()
opt_summary.columns=['experiment_id','variation_id','opt_first_date','opt_last_date','opt_guid_count']

# Add AB test duration col
opt_summary['duration'] = opt_summary.apply(lambda x: (x['opt_last_date'] - x['opt_first_date']).days,axis=1)

opt_summary

,experiment_id,variation_id,opt_first_date,opt_last_date,opt_guid_count,duration
0,20053754433,20041984016,2021-03-16 17:11:46.770,2021-04-21 23:59:04.760,340506,36
1,20053754433,20057713467,2021-03-16 17:11:46.861,2021-04-21 23:59:06.312,341096,36


In [108]:
# Add conversion count + conversion rate columns
opt_summary['opt_conv_count'] = opt_summary.apply(lambda x: count_conversions(x['experiment_id'],x['variation_id'],conv_event),axis=1)
opt_summary['opt_conv_rate'] = opt_summary.opt_conv_count / opt_summary.opt_guid_count
opt_summary

In [106]:
# Create df for datawarehouse info from function
dw_info = opt_summary.apply(lambda x : get_dw_info(x['experiment_id'],x['variation_id'],conv_event),axis=1)
dw_info = pd.DataFrame(dw_info.values.tolist(),columns=['dw_guid_count','dw_conv_count','dw_first_date','dw_last_date'])

get_dw_info_complete
get_dw_info_complete


In [107]:
summary = pd.concat([opt_summary,dw_info],axis=1)
summary['dw_conv_rate'] = summary.dw_conv_count / summary.dw_guid_count
summary

,experiment_id,variation_id,opt_first_date,opt_last_date,opt_guid_count,duration,opt_conv_count,opt_conv_rate,dw_guid_count,dw_conv_count,dw_first_date,dw_last_date,dw_conv_rate
0,20053754433,20041984016,2021-03-16 17:11:46.770,2021-04-21 23:59:04.760,340506,36,46153,0.135542,329000,49818,2021-03-16 00:00:00,2021-04-21 00:00:00,0.151422
1,20053754433,20057713467,2021-03-16 17:11:46.861,2021-04-21 23:59:06.312,341096,36,46370,0.135944,329637,49953,2021-03-16 00:00:01,2021-04-20 23:59:59,0.151539


### 4. Fishers Test


In [114]:
def fisher_exact(exp_id,guid_column,conv_column):    
    exp_summary = summary[summary.experiment_id==exp_id]
    exp_summary['churn_count'] = exp_summary[guid_column] - exp_summary[conv_column]
    
    contingency = pd.pivot_table(exp_summary,values=['churn_count',conv_column],columns=['variation_id']).to_numpy()
    
    oddsratio, pvalue = stats.fisher_exact(contingency)
    
    print('Contingency table: \n',contingency)
    print('Odds ratio: ',oddsratio)
    print('P-value: ',pvalue)
    
    if pvalue<0.05:
        print("\n Fisher exact test is significant for experiment #",exp_id)

In [112]:
fisher_exact(experiment_id,'dw_guid_count','dw_conv_count')

Contingency table:  [[279182 279684]
 [ 49818  49953]]
Odds ratio:  1.0009101172202115
P-value:  0.8960901271391716


In [113]:
fisher_exact(experiment_id,'opt_guid_count','opt_conv_count')

Contingency table:  [[294353 294726]
 [ 46153  46370]]
Odds ratio:  1.003430220140773
P-value:  0.6305577810267511
